# Predicting Airlines Delays
    
**Our goal in this project is to predict**:
- If a plane will arrive late or not

In the course of the project we will do some EDA on planes, airlines, plane manufacturers and delays using 
Posgresql, sklearn, Tableau and Flask.

    We are taking inspiration from the work of Russell Jurney - Agile Data Science 2.0 

In [1]:
########  Part 1 #######  data fetch and cleanup  #########  
#####  We are collecting:
#   Performance_2015.csv           :on time performance data from :
#    http://s3.amazonaws.com/agile_data_science/On_Time_On_Time_Performance_2015.cvs.bz2
#   faa_tail_number_inquiry.jsonl  :Plane information from : 
#   https://github.com/rjurney/Agile_Data_Code_2/tree/master/data/faa_tail_number_inquiry.jsonl


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)   # We can see up to 200 columns

In [3]:
!ls


Airlines logistic regression .ipynb  faa_tail_number_inquiry.jsonl
Airlines logistic regression1 .ipynb https.docx
AirlinesSQL.ipynb                    tail.csv
Performance_2015.csv                 tail1.csv
airlines.ipynb                       working_file
dataorigin.xlsx                      working_file.csv
delays1.csv


In [4]:
delays = pd.read_csv("./data/Performance_2015.csv")  # loading the 2015 flights in the US

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (69,76,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
delays.head(5) 

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2015,1,1,1,4,2015-01-01,AA,19805,AA,N001AA,1519,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,13244,1324402,33244,MEM,"Memphis, TN",TN,47,Tennessee,54,1345,1342.0,-3.0,0.0,0.0,-1.0,1300-1359,16.0,1358.0,1457.0,7.0,1510,1504.0,-6.0,0.0,0.0,-1.0,1500-1559,0.0,NaN,0.0,85.0,82.0,59.0,1.0,432.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,1,4,2015-01-01,AA,19805,AA,N001AA,1519,13244,1324402,33244,MEM,"Memphis, TN",TN,47,Tennessee,54,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1550,1546.0,-4.0,0.0,0.0,-1.0,1500-1559,9.0,1555.0,1712.0,9.0,1730,1721.0,-9.0,0.0,0.0,-1.0,1700-1759,0.0,NaN,0.0,100.0,95.0,77.0,1.0,432.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,1,4,2015-01-01,AA,19805,AA,N002AA,2349,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1845,1845.0,0.0,0.0,0.0,0.0,1800-1859,31.0,1916.0,2125.0,16.0,2115,2141.0,26.0,26.0,1.0,1.0,2100-2159,0.0,NaN,0.0,150.0,176.0,129.0,1.0,802.0,4,0.0,0.0,26.0,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,1,4,2015-01-01,AA,19805,AA,N003AA,1298,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,1820,2000.0,100.0,100.0,1.0,6.0,1800-1859,33.0,2033.0,2306.0,6.0,2120,2312.0,112.0,112.0,1.0,7.0,2100-2159,0.0,NaN,0.0,120.0,132.0,93.0,1.0,731.0,3,19.0,0.0,12.0,0.0,81.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,1,4,2015-01-01,AA,19805,AA,N003AA,1422,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,12094,1209402,34699,HDN,"Hayden, CO",CO,8,Colorado,82,800,918.0,78.0,78.0,1.0,5.0,0800-0859,30.0,948.0,1039.0,4.0,925,1043.0,78.0,78.0,1.0,5.0,0900-0959,0.0,NaN,0.0,145.0,145.0,111.0,1.0,769.0,4,78.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [6]:
list(delays) # This is used to get the column names to copy paste in the following cell

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'UniqueCarrier',
 'AirlineID',
 'Carrier',
 'TailNum',
 'FlightNum',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'FirstDepTime',
 'TotalAddGTime',
 

In [7]:
# remove columns from model data that is unnecessary - green keep
delays = delays.drop(
            [#'Year',
             #'Quarter',
             #'Month',
             #'DayofMonth',
             #'DayOfWeek',
             'FlightDate',
             #'UniqueCarrier',
             #'AirlineID',
             'Carrier',
             #'TailNum',
             #'FlightNum',
             #'OriginAirportID',
             'OriginAirportSeqID',
             #'OriginCityMarketID',
             #'Origin',
             'OriginCityName',
             'OriginState',
             #'OriginStateFips',
             #'OriginStateName',
             'OriginWac',
             #'DestAirportID',
             'DestAirportSeqID',
             #'DestCityMarketID',
             #'Dest',
             'DestCityName',
             'DestState',
             #'DestStateFips',
             #'DestStateName',
             'DestWac',
             #'CRSDepTime',
             #'DepTime',
             #'DepDelay',
             #'DepDelayMinutes',
             #'DepDel15',
             #'DepartureDelayGroups',
             'DepTimeBlk',
             'TaxiOut',
             'WheelsOff',
             'WheelsOn',
             'TaxiIn',
             #'CRSArrTime',
             #'ArrTime',
             #'ArrDelay',
             #'ArrDelayMinutes',
             #'ArrDel15',
             #'ArrivalDelayGroups',
             'ArrTimeBlk',
             #'Cancelled',
             'CancellationCode',
             'Diverted',
             'CRSElapsedTime',
             'ActualElapsedTime',
             #'AirTime',
             #'Flights',
             #'Distance',
             #'DistanceGroup',
             #'CarrierDelay',
             #'WeatherDelay',
             #'NASDelay',
             #'SecurityDelay',
             #'LateAircraftDelay',
             'FirstDepTime',
             'TotalAddGTime',
             'LongestAddGTime',
             'DivAirportLandings',
             'DivReachedDest',
             'DivActualElapsedTime',
             'DivArrDelay',
             'DivDistance',
             'Div1Airport',
             'Div1AirportID',
             'Div1AirportSeqID',
             'Div1WheelsOn',
             'Div1TotalGTime',
             'Div1LongestGTime',
             'Div1WheelsOff',
             'Div1TailNum',
             'Div2Airport',
             'Div2AirportID',
             'Div2AirportSeqID',
             'Div2WheelsOn',
             'Div2TotalGTime',
             'Div2LongestGTime',
             'Div2WheelsOff',
             'Div2TailNum',
             'Div3Airport',
             'Div3AirportID',
             'Div3AirportSeqID',
             'Div3WheelsOn',
             'Div3TotalGTime',
             'Div3LongestGTime',
             'Div3WheelsOff',
             'Div3TailNum',
             'Div4Airport',
             'Div4AirportID',
             'Div4AirportSeqID',
             'Div4WheelsOn',
             'Div4TotalGTime',
             'Div4LongestGTime',
             'Div4WheelsOff',
             'Div4TailNum',
             'Div5Airport',
             'Div5AirportID',
             'Div5AirportSeqID',
             'Div5WheelsOn',
             'Div5TotalGTime',
             'Div5LongestGTime',
             'Div5WheelsOff',
             'Div5TailNum',
              'Unnamed: 109']
            ,axis = 1 )

In [8]:
delays.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,UniqueCarrier,AirlineID,TailNum,FlightNum,OriginAirportID,OriginCityMarketID,Origin,OriginStateFips,OriginStateName,DestAirportID,DestCityMarketID,Dest,DestStateFips,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,Cancelled,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2015,1,1,1,4,AA,19805,N001AA,1519,11298,30194,DFW,48,Texas,13244,33244,MEM,47,Tennessee,1345,1342.0,-3.0,0.0,0.0,-1.0,1510,1504.0,-6.0,0.0,0.0,-1.0,0.0,59.0,1.0,432.0,2,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,1,4,AA,19805,N001AA,1519,13244,33244,MEM,47,Tennessee,11298,30194,DFW,48,Texas,1550,1546.0,-4.0,0.0,0.0,-1.0,1730,1721.0,-9.0,0.0,0.0,-1.0,0.0,77.0,1.0,432.0,2,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,1,4,AA,19805,N002AA,2349,13930,30977,ORD,17,Illinois,11298,30194,DFW,48,Texas,1845,1845.0,0.0,0.0,0.0,0.0,2115,2141.0,26.0,26.0,1.0,1.0,0.0,129.0,1.0,802.0,4,0.0,0.0,26.0,0.0,0.0
3,2015,1,1,1,4,AA,19805,N003AA,1298,11298,30194,DFW,48,Texas,10397,30397,ATL,13,Georgia,1820,2000.0,100.0,100.0,1.0,6.0,2120,2312.0,112.0,112.0,1.0,7.0,0.0,93.0,1.0,731.0,3,19.0,0.0,12.0,0.0,81.0
4,2015,1,1,1,4,AA,19805,N003AA,1422,11298,30194,DFW,48,Texas,12094,34699,HDN,8,Colorado,800,918.0,78.0,78.0,1.0,5.0,925,1043.0,78.0,78.0,1.0,5.0,0.0,111.0,1.0,769.0,4,78.0,0.0,0.0,0.0,0.0


In [9]:
mask = (delays['TailNum'] == 'N933EV')

In [10]:
delays[mask]

,Year,Quarter,Month,DayofMonth,DayOfWeek,UniqueCarrier,AirlineID,TailNum,FlightNum,OriginAirportID,OriginCityMarketID,Origin,OriginStateFips,OriginStateName,DestAirportID,DestCityMarketID,Dest,DestStateFips,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,Cancelled,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
5522,2015,1,1,1,4,EV,20366,N933EV,4928,14108,34108,PIA,17,Illinois,10397,30397,ATL,13,Georgia,605,600.0,-5.0,0.0,0.0,-1.0,901,837.0,-24.0,0.0,0.0,-2.0,0.0,83.0,1.0,565.0,3,NaN,NaN,NaN,NaN,NaN
5523,2015,1,1,1,4,EV,20366,N933EV,4985,10397,30397,ATL,13,Georgia,10781,30781,BTR,22,Louisiana,1400,1356.0,-4.0,0.0,0.0,-1.0,1438,1430.0,-8.0,0.0,0.0,-1.0,0.0,75.0,1.0,448.0,2,NaN,NaN,NaN,NaN,NaN
5524,2015,1,1,1,4,EV,20366,N933EV,4985,10781,30781,BTR,22,Louisiana,10397,30397,ATL,13,Georgia,1503,1504.0,1.0,1.0,0.0,0.0,1729,1722.0,-7.0,0.0,0.0,-1.0,0.0,62.0,1.0,448.0,2,NaN,NaN,NaN,NaN,NaN
5525,2015,1,1,1,4,EV,20366,N933EV,5069,10397,30397,ATL,13,Georgia,11612,31612,EVV,18,Indiana,952,943.0,-9.0,0.0,0.0,-1.0,1015,1006.0,-9.0,0.0,0.0,-1.0,0.0,67.0,1.0,350.0,2,NaN,NaN,NaN,NaN,NaN
5526,2015,1,1,1,4,EV,20366,N933EV,5069,11612,31612,EVV,18,Indiana,10397,30397,ATL,13,Georgia,1040,1035.0,-5.0,0.0,0.0,-1.0,1304,1252.0,-12.0,0.0,0.0,-1.0,0.0,59.0,1.0,350.0,2,NaN,NaN,NaN,NaN,NaN
5527,2015,1,1,1,4,EV,20366,N933EV,5424,10397,30397,ATL,13,Georgia,10980,30980,CHA,47,Tennessee,1900,1856.0,-4.0,0.0,0.0,-1.0,1945,1941.0,-4.0,0.0,0.0,-1.0,0.0,23.0,1.0,106.0,1,NaN,NaN,NaN,NaN,NaN
32497,2015,1,1,11,7,EV,20366,N933EV,5022,10397,30397,ATL,13,Georgia,12323,32323,ILM,37,North Carolina,2054,2056.0,2.0,2.0,0.0,0.0,2212,2204.0,-8.0,0.0,0.0,-1.0,0.0,51.0,1.0,377.0,2,NaN,NaN,NaN,NaN,NaN
32498,2015,1,1,11,7,EV,20366,N933EV,5023,10397,30397,ATL,13,Georgia,10980,30980,CHA,47,Tennessee,1330,1326.0,-4.0,0.0,0.0,-1.0,1411,1407.0,-4.0,0.0,0.0,-1.0,0.0,27.0,1.0,106.0,1,NaN,NaN,NaN,NaN,NaN
32499,2015,1,1,11,7,EV,20366,N933EV,5023,10980,30980,CHA,47,Tennessee,10397,30397,ATL,13,Georgia,1436,1430.0,-6.0,0.0,0.0,-1.0,1533,1539.0,6.0,6.0,0.0,0.0,0.0,26.0,1.0,106.0,1,NaN,NaN,NaN,NaN,NaN
32500,2015,1,1,11,7,EV,20366,N933EV,5166,10781,30781,BTR,22,Louisiana,10397,30397,ATL,13,Georgia,1005,957.0,-8.0,0.0,0.0,-1.0,1230,1223.0,-7.0,0.0,0.0,-1.0,0.0,67.0,1.0,448.0,2,NaN,NaN,NaN,NaN,NaN


In [11]:
delays.to_csv('./data/delays1.csv',index=False) # without index=false it creates an index column

In [12]:
df1 = pd.read_json('./data/faa_tail_number_inquiry.jsonl',lines=True)

In [13]:
df1.head()

,TailNum,engine_manufacturer,engine_model,manufacturer,mfr_year,model,owner,owner_state,serial_number
0,N933EV,GE,CF34 SERIES,BOMBARDIER INC,2005,CL-600-2B19,DELTA AIR LINES INC,GEORGIA,8022
1,N917WN,CFM INTL,CFM56-7B24,BOEING,2008,737-7H4,SOUTHWEST AIRLINES CO,TEXAS,36624
2,N438WN,CFM INTL.,CFM56 SERIES,BOEING,2003,737-7H4,SOUTHWEST AIRLINES CO,TEXAS,29833
3,N283VA,CFM INTL,CFM56-5B4/3,AIRBUS,2015,A320-214,VIRGIN AMERICA INC,CALIFORNIA,6787
4,N473UA,IVCHENKO,AL-25SERIES,AIRBUS INDUSTRIE,2001,A320-232,UNITED AIRLINES INC,ILLINOIS,1469


In [28]:
list(df1.dtypes) # This is used to get the column names to copy paste in the following cell

[dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O')]

In [15]:
# remove columns from model data that is unnecessary - green keep
df1 = df1.drop([#'TailNum',
                 'engine_manufacturer',
                 'engine_model',
                 #'manufacturer',
                 #'mfr_year',
                 #'model',
                 'owner',
                 'owner_state',
                 #'serial_number',
                ],axis = 1 )

In [16]:
df1.head()

,TailNum,manufacturer,mfr_year,model,serial_number
0,N933EV,BOMBARDIER INC,2005,CL-600-2B19,8022
1,N917WN,BOEING,2008,737-7H4,36624
2,N438WN,BOEING,2003,737-7H4,29833
3,N283VA,AIRBUS,2015,A320-214,6787
4,N473UA,AIRBUS INDUSTRIE,2001,A320-232,1469


In [17]:
df1.to_csv('./data/tail1.csv',index=False)   # savings the file before loading them in sql

In [ ]:
########  END OF Part 1 #######  data fetch and cleanup  #########  